In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.metrics import recall_score, confusion_matrix
from itertools import combinations
from sklearn.model_selection import KFold
from numpy import array

In [2]:
classes    = ['clear', 'mask']
feature_sets=['_ComParE','BoAW','auDeep','DeepSpectrum_resnet50','FV6','FV_LLD']
tuned_parameters = {'gamma': [100,10,1,1e-1,1e-2,1e-3],
                    'C': [100,10,1,1e-1,1e-2,1e-3]}

In [3]:
def readcsv(feature):
    files=os.listdir()
    for file in files:
        if feature in file:
            return pd.read_csv(file)['prob0'].values
    

In [4]:
predlists=[]
for i in range(len(feature_sets)):
    print(feature_sets[i])
    print(readcsv(feature_sets[i]))
    predlists.append([])
    predlists[-1].append(readcsv(feature_sets[i]))
    predlists[-1].append(1-readcsv(feature_sets[i]))
label_file    = 'labels.csv'
df_labels = pd.read_csv(label_file)
y_devel = df_labels['label'][df_labels['file_name'].str.startswith('devel')].values

pred=[]
for i in range(len(predlists[0][0])):
    pred.append([predlists[j][0][i] for j in range(len(predlists))])


_ComParE
[0.56543286 0.81283127 0.14612692 ... 0.28267454 0.64667312 0.24305277]
BoAW
[0.34299999 0.9256225  0.28748837 ... 0.52506372 0.59952203 0.4729858 ]
auDeep
[0.63642953 0.86604398 0.19370036 ... 0.34671974 0.31651106 0.34988477]
DeepSpectrum_resnet50
[0.41269561 0.77586207 0.43334282 ... 0.07522409 0.72937518 0.37582475]
FV6
[0.64576268 0.96587454 0.06827503 ... 0.22243974 0.340786   0.48516783]
FV_LLD
[7.81021898e-01 1.00000000e+00 9.46969697e-04 ... 9.46969697e-04
 9.42982456e-01 8.38323353e-03]


In [7]:
def fusion(feature_sets):
    
    #[[[0],[1]],[[0],[1]]]
    predlists=[]
    for i in range(len(feature_sets)):
        predlists.append([])
        predlists[-1].append(readcsv(feature_sets[i]))
        predlists[-1].append(1-readcsv(feature_sets[i]))
    label_file    = 'labels.csv'
    df_labels = pd.read_csv(label_file)
    y_devel = df_labels['label'][df_labels['file_name'].str.startswith('devel')].values
    
    pred=[]
    for i in range(len(predlists[0][0])):
        pred.append([predlists[j][0][i] for j in range(len(predlists))])
        
    uar_scores = []
    X_train=pred
    print('gamma:\t',end='')
    for g in tuned_parameters['gamma']:
        print(g,'\t',end='')
    print('\nC:')
    for comp in tuned_parameters['C']:
        print(comp,':\t',end='')
        for g in tuned_parameters['gamma']:
            clf = svm.SVC(C=comp,gamma=g)
        
            #cross validation
            kf = KFold(n_splits=5)
            uarList=[]
            for train_index, test_index in kf.split(X_train):
                #print("TRAIN:", train_index, "TEST:", test_index)
                clf.fit([X_train[i] for i in train_index], [y_devel[i] for i in train_index])
                y_pred = clf.predict([X_train[i] for i in test_index])
                uarList.append(recall_score([y_devel[i] for i in test_index], y_pred, labels=classes, average='macro'))
                
            uar_scores.append(np.mean(uarList))
            
            print('{0:.3f}'.format(uar_scores[-1]*100),'\t',end='')
        print()


    return max(uar_scores)

In [7]:
fusion(feature_sets)

gamma:	100 	10 	1 	0.1 	0.01 	0.001 	
C:
100 :	63.1 	66.1 	71.2 	71.1 	70.3 	71.3 	
10 :	63.9 	68.8 	71.1 	71.2 	71.2 	71.2 	
1 :	67.2 	70.9 	71.1 	70.6 	71.1 	70.3 	
0.1 :	52.6 	71.1 	71.1 	70.9 	70.3 	50.0 	
0.01 :	50.0 	70.9 	70.9 	70.3 	50.0 	50.0 	
0.001 :	50.0 	50.0 	70.2 	50.0 	50.0 	50.0 	


0.7125423556535972